In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='https://revistavectoresdeinvestigacion.com/'
url_root= 'https://revistavectoresdeinvestigacion.com/'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
box = soup.find('div', class_='entry-content')
volumen=box.findAll('a', rel='noreferrer noopener')
vol = [x.get('href')for x in volumen]
#vol=[urljoin(url_root,i) for i in vol]
vol

In [ ]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='entry-content')
    volumen=box.findAll('a', rel='noreferrer noopener')
    vol = [x.get('href')for x in volumen]
    #vol=[urljoin(url_root,i) for i in vol]
    return vol

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Revista Vectores de Investigación'
    a='Multidisciplinaria'
    te=None
    urrl='https://www.revistavectoresdeinvestigacion.com/'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.findAll('a', rel='noreferrer noopener').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', class_='description').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('section', class_='item abstract').get_text(strip=True)
        content_book['Abstract']=de.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=None
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
        #obtener link PDF
    try:
        ancla_link=url
        content_book['Link PDF']=ancla_link
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:16]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df=df_catalogo[~ df_catalogo["Link PDF"].isin(["http://www.editorial-apublicaciones.com","https://adcale.org/","https://fundacionactforum.org/iled",
"https://fundacionactforum.org/cieic","https://fundacionactforum.org/"])]
df

In [ ]:
df.to_csv('Revista212.csv', index=False)